In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [3]:
from textsummarizer.constants import *
from textsummarizer.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [4]:
from urllib import request
import zipfile
import os
from textsummarizer.logging import logger
from textsummarizer.utils.common_func import get_size
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


    

/home/sachin/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-22 15:16:55,813: INFO:  config:  PyTorch version 2.7.1 available.]


In [5]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_tokens(self, example_batch):

        input_encodings = self.tokenizer(example_batch['dialogue'], truncation=True, max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], truncation=True, max_length=128)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert_data(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_temp = data_samsum.map(self.convert_examples_to_tokens, batched=True)
        data_samsum_temp.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [6]:

data_samsum = load_dataset('artifacts/data_ingestion/samsum_dataset')



#print("Sample example:", data_samsum['train'][0])
print(data_samsum['train'].column_names)  


    

Generating train split: 1 examples [00:00, 493.85 examples/s]
Generating validation split: 1 examples [00:00, 774.14 examples/s]
Generating test split: 1 examples [00:00, 860.19 examples/s]

['_data_files', '_fingerprint', '_format_columns', '_format_kwargs', '_format_type', '_output_all_columns', '_split']


In [7]:
try:
    config = ConfigurationManager()
    config_data_transformation = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = config_data_transformation)
    data_transformation.convert_data()
except Exception as e:
    raise e

[2025-06-22 15:16:56,014: INFO:  common_func:  Yaml file: config/config.yaml has been loaded succesfully]
[2025-06-22 15:16:56,016: INFO:  common_func:  Yaml file: params.yaml has been loaded succesfully]
[2025-06-22 15:16:56,017: INFO:  common_func:  created directory at: artifacts]
[2025-06-22 15:16:56,019: INFO:  common_func:  created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/sachin/miniconda3/envs/myenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 174071.06 examples/s]
